In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import timedelta, date
import time

# Events

In [14]:
def grab_events_data(start_year, start_month, start_day, end_year, end_month, end_day):
    month_str = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    month_num = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    def daterange(date1, date2):
        for n in range(int ((date2 - date1).days)+1):
            yield date1 + timedelta(n)

    start_dt = date(start_year, start_month, start_day)
    end_dt = date(end_year, end_month, end_day)
    datelist = []
    for dt in daterange(start_dt, end_dt):
        datelist.append(dt.strftime("%Y-%m-%d"))
    
    for i in range(len(datelist)):
        web_url = 'https://finance.yahoo.com/calendar/economic?day=' +  datelist[i]
        page = requests.get(web_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        a = soup.find_all('td')
        if a == []:
            dataset = []
            df = pd.DataFrame(dataset, columns=['Event', 'Event Time', 'For', 'Actual', 'Market Expectation', 
                                                'Country', 'Prior to This', 'Revised from'])
        else:
            raw_data = []
            for j in range(len(a)):
                raw_data.append(a[j].string)

            dataset = np.split(np.array(raw_data), len(raw_data)/8)
            df = pd.DataFrame(dataset, columns=['Event', 'Event Time', 'For', 'Actual', 'Market Expectation', 
                                       'Country', 'Prior to This', 'Revised from'])
            

            for a in range(len(df['For'])):
                for b in range(len(month_str)):
                    if df['For'][a] == month_str[b]:
                        df['For'][a] = '=' + '"' + month_num[b] + '/2017' + '"'
#         df.to_csv('b.csv', date_format='%m/%Y')
        filename = datelist[i] + '.csv'
        df.to_csv(filename)
#     print(datelist)

In [15]:
grab_events_data(2017,1,1,2017,1,10)

['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10']


# Stock price

In [ ]:
def grab_hist_stock_data(symbol, start_year, start_month, start_day, end_year, end_month, end_day):
    
    start_dt = date(start_year, start_month, start_day)
    end_dt = date(end_year, end_month, end_day)
    start_str = str(int(time.mktime(start_dt.timetuple())))
    end_str = str(int(time.mktime(end_dt.timetuple())))
    
    web_url = 'https://finance.yahoo.com/quote/' +  symbol + '/history?period1=' + start_str + '&period2=' + end_str + '&interval=1d&filter=history&frequency=1d'
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    a = soup.find_all('td')
    
    df_columns = ['Date', 'Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume']
    if a == []:
        dataset = []
        df = pd.DataFrame(dataset, columns=df_columns)
    else:
        raw_data = []
        for j in range(len(a)):
            if a[j].string == None and j != len(a)-1:
                raw_data = raw_data[:-1]
            elif a[j].string == None:
                pass
            else:
                raw_data.append(a[j].string)

        dataset = np.split(np.array(raw_data), len(raw_data)/7)
        df = pd.DataFrame(dataset, columns=df_columns)
        filename = symbol + '_' + start_dt.strftime("%Y-%m-%d") + '_' + end_dt.strftime("%Y-%m-%d") + '.csv'
        df.to_csv(filename)

In [ ]:
grab_hist_stock_data('GOOG', 2017,1,1,2017,10,10)